In [1]:
from typing import Optional, Union, List
from datetime import date, datetime

from pydantic import BaseModel, Field

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.tools import StructuredTool
from langchain_google_vertexai import ChatVertexAI

In [2]:
def object_to_xml(data: Union[dict, bool], root='object', ignore=None):
    if not ignore:
        ignore = []
    elif isinstance(ignore, str):
        ignore = [ignore]
    xml = f'<{root}>'
    if isinstance(data, dict):
        for key, value in data.items():
            if not(key in ignore):
                xml += object_to_xml(value, key, ignore)

    elif isinstance(data, (list, tuple, set)):
        for item in data:
            xml += object_to_xml(item, 'item', ignore)

    else:
        xml += str(data)

    xml += f'</{root}>'
    return xml

In [3]:
class Task(BaseModel):
    id: Optional[int] = None
    due: date
    description: str
    status: int = 0

class TaskList(BaseModel):
    last_update: datetime = Field(default_factory=datetime.now)
    tasks: List[Task] = Field(default = [])

    def __str__(self):
        return object_to_xml(self.model_dump()['tasks'], 'task')
    
    def add(self, due: date, description: str, status: int = 0):
        if len(self.tasks) > 0:
            id = max([t.id for t in self.tasks]) + 1
        else:
            id = 1
        self.tasks.append(Task(id = id, due = due, description = description, status = status))

In [4]:
l = TaskList()
l.add(date(2024, 10, 22), 'submit chemistry homework')
l.add(date(2024, 10, 23), 'second chapter in chemistry texbook')

In [5]:
llm = ChatVertexAI(model='gemini-1.5-flash-002')

In [6]:
prompt = ChatPromptTemplate([
    ("system", "You are an executive asistant for {user}. Today's date is {date}. Make your response concise and direct to the point"),
    ("user", "My task list is {tasks}. Let's start with what my today's tasks are")
])

In [7]:
messages = prompt.invoke({'user':'high school student', 'date':'Oct 21, 2024', 'tasks': str(l)}).messages

In [8]:
response = llm.invoke(messages)

In [9]:
response.content

"Today's tasks: None.  Focus on chemistry homework (due tomorrow).\n"